In [ ]:
from pyspark.sql.functions import col, trim, upper, when, lit, current_timestamp, \
                                  substring, regexp_replace, unix_timestamp, \
                                  year, month, dayofmonth, dayofweek, avg, min, max, \
                                  floor, hour, date_format, desc, lag, abs, \
                                  lead, datediff, when, window, mean, stddev, skewness, kurtosis, \
                                  sum, to_date, count, concat, countDistinct \

from pyspark.sql.window import Window
from pyspark.ml.feature import QuantileDiscretizer

from pyspark.sql.types import StringType, IntegerType, DoubleType, LongType, TimestampType

import plotly.express as px
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

from pyspark.sql import functions as F
#--- Rutas de datos ---
bronze_path = "Files/bronze/transacciones_servicios/"
silver_path = "Files/silver/transacciones_servicios/"

# --- Parametros de Gobernanza ---
notebook_version = "1.0"

StatementMeta(, 209a79ae-d4de-484b-878b-39d462c7e58c, 62, Finished, Available, Finished)

In [ ]:
print("Iniciando la lectura de la tabla de transacciones desde la zona Bronze...")

# Leemos la tabla Delta desde Bronze
# Para mejorar el rendimiento en producción, se recomienda leer particiones específicas
df_bronze = spark.read.format("delta").load(bronze_path)

print(f"Lectura completada. Se leyeron {df_bronze.count()} registros.")
df_bronze.printSchema()

StatementMeta(, 209a79ae-d4de-484b-878b-39d462c7e58c, 63, Finished, Available, Finished)

Iniciando la lectura de la tabla de transacciones desde la zona Bronze...
Lectura completada. Se leyeron 24931310 registros.
root
 |-- ACCOUNT_NBR: long (nullable = true)
 |-- IDENT: string (nullable = true)
 |-- POSTING_DATE: timestamp (nullable = true)
 |-- PRODUCT_TYPE_ID: integer (nullable = true)
 |-- DESCRIPTION1: string (nullable = true)
 |-- DESCRIPTION2: string (nullable = true)
 |-- DESCRIPTION3: string (nullable = true)
 |-- AMOUNT: double (nullable = true)
 |-- CASHIER_NBR: integer (nullable = true)
 |-- TO_ACCOUNT_NBR: long (nullable = true)
 |-- IDSEGMENTO: integer (nullable = true)



In [ ]:
# Obtener todas las columnas de tipo string para aplicar limpieza
string_columns = [item[0] for item in df_bronze.dtypes if item[1].startswith('string')]

df_silver = df_bronze
# Estandarizar valores nulos (reemplazar "." por NULL)
detail_columns = ["DESCRIPTION1", "DESCRIPTION2", "DESCRIPTION3"]
for detail_col in detail_columns:
    df_silver = df_silver.withColumn(detail_col, when(col(detail_col) == ".", None).otherwise(col(detail_col)))

StatementMeta(, 209a79ae-d4de-484b-878b-39d462c7e58c, 64, Finished, Available, Finished)

In [ ]:
print("Aplicando reglas de enriquecimiento y transformación...")

# Crear Dimensiones de Tiempo
df_silver = df_silver.withColumn("ANIO_TRANSACCION", year(col("POSTING_DATE")))
df_silver = df_silver.withColumn("MES_TRANSACCION", month(col("POSTING_DATE")))
df_silver = df_silver.withColumn("DIA_TRANSACCION", dayofmonth(col("POSTING_DATE")))
df_silver = df_silver.withColumn("DIA_SEMANA", dayofweek(col("POSTING_DATE")))

StatementMeta(, 209a79ae-d4de-484b-878b-39d462c7e58c, 65, Finished, Available, Finished)

Aplicando reglas de enriquecimiento y transformación...


In [ ]:
print("Aplicando renombrado de columnas y añadiendo metadatos...")

# 1. Renombrar columnas a snake_case
new_column_names = {c: c.lower() for c in df_silver.columns}
df_silver_renamed = df_silver.select([col(c).alias(new_column_names.get(c, c)) for c in df_silver.columns])

# 2. Añadir metadatos del proceso
df_final = df_silver_renamed.withColumn("fecha_proceso_silver", current_timestamp())
df_final = df_final.withColumn("version_notebook", lit(notebook_version))

print("Schema final de la tabla Silver:")
df_final.printSchema()

StatementMeta(, 209a79ae-d4de-484b-878b-39d462c7e58c, 66, Finished, Available, Finished)

Aplicando renombrado de columnas y añadiendo metadatos...
Schema final de la tabla Silver:
root
 |-- account_nbr: long (nullable = true)
 |-- ident: string (nullable = true)
 |-- posting_date: timestamp (nullable = true)
 |-- product_type_id: integer (nullable = true)
 |-- description1: string (nullable = true)
 |-- description2: string (nullable = true)
 |-- description3: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- cashier_nbr: integer (nullable = true)
 |-- to_account_nbr: long (nullable = true)
 |-- idsegmento: integer (nullable = true)
 |-- anio_transaccion: integer (nullable = true)
 |-- mes_transaccion: integer (nullable = true)
 |-- dia_transaccion: integer (nullable = true)
 |-- dia_semana: integer (nullable = true)
 |-- fecha_proceso_silver: timestamp (nullable = false)
 |-- version_notebook: string (nullable = false)



In [ ]:
df_final = df_final.withColumn("amount", col("amount").cast(DoubleType())) \
                   .withColumn("posting_date", date_format("posting_date", "yyyy-MM-dd")) \
                   .withColumn("nombre_dia_semana", date_format(col("posting_date"), 'E'))

StatementMeta(, 209a79ae-d4de-484b-878b-39d462c7e58c, 67, Finished, Available, Finished)

In [ ]:
df_final.printSchema()

StatementMeta(, 209a79ae-d4de-484b-878b-39d462c7e58c, 68, Finished, Available, Finished)

root
 |-- account_nbr: long (nullable = true)
 |-- ident: string (nullable = true)
 |-- posting_date: string (nullable = true)
 |-- product_type_id: integer (nullable = true)
 |-- description1: string (nullable = true)
 |-- description2: string (nullable = true)
 |-- description3: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- cashier_nbr: integer (nullable = true)
 |-- to_account_nbr: long (nullable = true)
 |-- idsegmento: integer (nullable = true)
 |-- anio_transaccion: integer (nullable = true)
 |-- mes_transaccion: integer (nullable = true)
 |-- dia_transaccion: integer (nullable = true)
 |-- dia_semana: integer (nullable = true)
 |-- fecha_proceso_silver: timestamp (nullable = false)
 |-- version_notebook: string (nullable = false)
 |-- nombre_dia_semana: string (nullable = true)



In [ ]:
"""
Vamos a crear una variable que nos permita identificar la diferencia de dias entre transacciones realizadas
por un cliente a un mismo beneficiario.

En este caso vamos a usar una funcion de ventana usando la particion de cuenta origen y cuenta destino ordenando
por la fecha de la transacccion y contanto la cantidad de dias entre cada ventana
"""

window_spec = Window.partitionBy("account_nbr", "to_account_nbr").orderBy("posting_date")

# 2. Calcular diferencia en días y en monto con la transacción anterior y la siguiente
df_final = df_final.withColumn("monto_anterior", lag("amount", 1).over(window_spec)) \
                   .withColumn("fecha_anterior", lag("posting_date", 1).over(window_spec))\
                   .withColumn("fecha_siguiente_transaccion",lead("posting_date", 1).over(window_spec))\
                   .withColumn("monto_siguiente_transaccion",lead("amount", 1).over(window_spec)
)

# calculamos la cantidad de dias de diferencia entre las transaccioes
df_final = df_final.withColumn("dias_diferencia_anterior_transaccion", datediff(col("posting_date"), col("posting_date"))) \
                   .withColumn("diferencia_monto_anterior", abs(col("amount") - col("monto_anterior")))\
                   .withColumn("dias_diferencia_siguiente_transaccion",datediff(col("fecha_siguiente_transaccion"), col("posting_date")))\
                   .withColumn("diferencia_monto_siguiente", abs(col("amount") - col("monto_siguiente_transaccion")))

StatementMeta(, 209a79ae-d4de-484b-878b-39d462c7e58c, 69, Finished, Available, Finished)

In [ ]:
df_final.printSchema()

StatementMeta(, 209a79ae-d4de-484b-878b-39d462c7e58c, 70, Finished, Available, Finished)

root
 |-- account_nbr: long (nullable = true)
 |-- ident: string (nullable = true)
 |-- posting_date: string (nullable = true)
 |-- product_type_id: integer (nullable = true)
 |-- description1: string (nullable = true)
 |-- description2: string (nullable = true)
 |-- description3: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- cashier_nbr: integer (nullable = true)
 |-- to_account_nbr: long (nullable = true)
 |-- idsegmento: integer (nullable = true)
 |-- anio_transaccion: integer (nullable = true)
 |-- mes_transaccion: integer (nullable = true)
 |-- dia_transaccion: integer (nullable = true)
 |-- dia_semana: integer (nullable = true)
 |-- fecha_proceso_silver: timestamp (nullable = false)
 |-- version_notebook: string (nullable = false)
 |-- nombre_dia_semana: string (nullable = true)
 |-- monto_anterior: double (nullable = true)
 |-- fecha_anterior: string (nullable = true)
 |-- fecha_siguiente_transaccion: string (nullable = true)
 |-- monto_siguiente_transaccio

In [ ]:
df_final = df_final.filter(~(col("description1")).rlike("Recargas|(Recargas)|Recarga|(Recarga)|Master|Prestige|Medico|Medica|Medicoop|Internet|Edgar|Herminia|Casa|Pres|Coopmssa|Coop leal|COOP|Coop|Royal|ACH|TRAN|INFOTEP|Royal Prestige|( Recarga )|(RECARGA)|RECARGA|TSS|DGII|DGA"))

StatementMeta(, 209a79ae-d4de-484b-878b-39d462c7e58c, 81, Finished, Available, Finished)

In [ ]:
display(df_final)

StatementMeta(, 209a79ae-d4de-484b-878b-39d462c7e58c, 82, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 003e2269-be3d-465d-8a72-cc90bd8ec9d6)

In [ ]:
print("Iniciando escritura de los datos transformados en la zona Silver...")

# La estrategia de partición se mantiene igual que en Bronze para consistencia (usando la nueva columna renombrada)
df_final.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(silver_path)

print(f"Proceso completado exitosamente. Los datos han sido guardados en: {silver_path}")

StatementMeta(, 209a79ae-d4de-484b-878b-39d462c7e58c, 84, Finished, Available, Finished)

Iniciando escritura de los datos transformados en la zona Silver...


Proceso completado exitosamente. Los datos han sido guardados en: Files/silver/transacciones_servicios/
